In [14]:
# Import relevant packages
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [25]:
dataset = []
fns = []
for line in open('dataset-example.csv'):
    fn, txt = line.split('|')
    fns.append(fn)
    dataset.append(txt)


In [41]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(dataset)
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(dataset)

In [42]:
X_tfidf
feature_names = tf.get_feature_names()
feature_index = tfidf_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])

NameError: name 'doc' is not defined

In [39]:
feature_names[-100:]

['supplementalservicingai',
 'sync',
 'sys',
 'system',
 'system32',
 'systemapps',
 'systemresources',
 'systems',
 'syswow64',
 'table',
 'tables',
 'tag',
 'target',
 'target_netbios_name',
 'targetfilelength',
 'text',
 'the',
 'thumbnail',
 'tifilefetcher',
 'time',
 'timestamps',
 'tm',
 'tm_31bf3856ad364e35_10',
 'tmlrendering',
 'tocolimplementation_31bf3856ad364e35_11',
 'tool',
 'trc',
 'tsworkspace',
 'tws',
 'txf',
 'txt',
 'ty',
 'type',
 'ui',
 'ultimate',
 'undefined',
 'unit',
 'units',
 'unix',
 'unknown',
 'unsigned',
 'up',
 'update',
 'urlmon',
 'user',
 'users',
 'utf',
 'utilities',
 'v4',
 'v6iygfzueqiqxhbmgcaijuezddwhsayecyejkgggbrugdq4p',
 'vadsharedvoiceagents',
 'value',
 'values',
 'vch',
 'vendor',
 'version',
 'vert',
 'viab',
 'vnd',
 'voiceagentscommon',
 'volumelabel',
 'volumeserial',
 'volumetype',
 'wanarp',
 'wbemcore',
 'wcp',
 'web',
 'webapi_31bf3856ad364e35_10',
 'webcheck',
 'webplatstorageserver',
 'weighted',
 'wer',
 'white',
 'width',
 'win